# **Assignment 2 Manual (From Scratch)**

- **Programmers:**
  - Shaun Pritchard
  - Ismael A Lopez
- **Date:** 11-8-2021
- **Assignment:** 2
- **Prof:** M.DeGiorgio

<hr>

### **Overview: Secondary - Assignment 2**

We analyzed the credit card data from N=400 training observations that you examined in Programming Assignment 1 using a penalized (regularized) least squares fit of a linear model using elastic net, with model parameters obtained by coordinate descent. 

Initially, we each worked independently, then we collaborated afterwards to finalize the assignment deliverables. This resulted in the completion of 2 methods for achieving the same goal, namely implementing ElastNet with coordinate descent. This is the second take on assignment 2. The aim was to display different methods of achieving the same abstraction.


## **Import data**

In [ ]:
#Math libs
from math import sqrt
from scipy import stats
import os
# Data Science libs
import numpy as np
import pandas as pd
# Graphics libs
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#Timers
!pip install pytictoc
from pytictoc import TicToc

## **Import Data**

In [ ]:
# Import Data
df = pd.read_csv('/content/Credit_N400_p9.csv')

In [ ]:
# Validate data import
df.head(3)

## **Data Pre Proccessing**

In [ ]:
# Assign dummy variables to catigorical feature attributes
df = df.replace({'Male': 0, 'Female':1, 'No': 0, 'Yes': 1})
df.head(3)

In [ ]:
# separate the predictors from the response
X = df.to_numpy()[:, :-1]
Y = df.to_numpy()[:, -1]
print('Convert dataframe to numpy array:', X.shape, Y.shape)

## **Set Global Variables**

In [ ]:
# Set local variables
# 9-Tuning Parms
λ  = [1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]

# 6 learning & convergence rate
α =  [0, 0.2, 0.4, 0.6, 0.8, 1]

# K-folds
k = 5

#Iterations
n_iters = 1000 # itterations

#log base of lambda
λ_log = np.log10(λ) 

# Set verbose to True
verbose = True

# Set n x m matrix variable
X_p = X

# Set n vector variable
Y_p  = Y

## **Instantiate Data**

In [ ]:
# Randomize N x M and N data
def randomize_data(X_p, Y_p):
  matrix = np.concatenate((X_p, Y_p[:, None]), 1)
  np.random.shuffle(matrix)
  return matrix[:, :-1], matrix[:, -1]

In [ ]:
# Initilize random sample data
x, y = randomize_data(X_p, Y_p)

In [ ]:
# Set Global variable for samples and number of X = N X M features
X1 = x.shape[0]
X2 = x.shape[1]

In [ ]:
# Create a 𝛽 matrix to store the predictors 
𝛽 = np.zeros([k, len(λ), len(α), X2])

In [ ]:
# Store 5 K-fold cross validation results 
CV = np.zeros([k, len(λ), len(α)])

In [ ]:
# Compute the number of validation test samples and indices  based on k-folds 
test_x = X1 // k 
test_i = list(range(0, X1, test_x))
        
if True:
    print('Implemnting {} training of {} test validation samples for each 5-k CV fold.'.format(
        X1 - test_x, test_x)
    )

## **Implment Functions**

In [ ]:
# Standardize  X
def standardize(x, mean_x, std_x):
  return (x - mean_x) / std_x 

In [ ]:
# Center response variables
def centerResponses(y, mean):
  return y - mean

In [ ]:
# predicit x
def predict(x):
  x = standardize(x, mean_x, std_x)
  return np.matmul(x, 𝛽x)

In [ ]:
# Calculate MSE score 
def score(x_test, y_test,  𝛽x):
  ŷ = np.matmul(x_test, 𝛽x)
  mse = np.mean((y_test - ŷ) ** 2)
  return mse

### **Coordinate Descent Algortihm**

In [ ]:
# implement Coordinate Descent
def coordinateDescent(x, y, 𝛽x, sum_sq, lamb, alpha):
  for k in range(X2):
    # RSS minus the k coefficient 
    RSS = y - np.matmul(x, 𝛽x) + (x[:, k] * 𝛽x[k])[:, None]
            
    # Calualte the RSS Loss function
    a_k = np.matmul(x[:, k].T, RSS)[0]
            
    # update B_k
    𝛽k = np.absolute(a_k) - lamb * (1 - alpha) / 2
    𝛽k = 𝛽k if 𝛽k >= 0 else 0
    𝛽x[k, 0] = np.sign(a_k) * 𝛽k / (sum_sq[k] + lamb * alpha)

  return 𝛽x

### **Elastic Net - Cross Validation Algorithm**

In [ ]:
for i_lambda, lamb in enumerate(λ): # loop through λ lambda
    for i_alpha, alpha in enumerate(α): # loop through α
        for i_fold, i_test in zip(range(k), test_i): # loop through folds

            # Validates and trains the CV iteration based on the validation and training sets.     
            x_test = x[i_test:i_test + test_x]
            x_train = np.delete(x, np.arange(i_test, i_test + test_x), axis = 0)
            y_test = y[i_test:i_test + test_x]
            y_train = np.delete(y, np.arange(i_test, i_test + test_x), axis = 0)


            # Standardize x and center y  5 K-fold trianing and test data
            mean_x, std_x = np.mean(x_train, 0), np.std(x_train, 0)
            mean_res = np.mean(y_train)

            # X training and test
            x_train = standardize(x_train, mean_x, std_x)
            x_test = standardize(x_test, mean_x, std_x)
             
            # Y training and test 
            y_train = centerResponses(y_train, mean_res)[:, None]
            y_test = centerResponses(y_test, mean_res)[:, None]
                      
            # compute b_k given this fold 
            sum_sq = np.sum(x_train ** 2, 0)
          

            # initialize random Beta for this lambda and fold
            𝛽x = np.random.uniform(low = -1, high = 1, size = (X2, 1))

            # Iterate 1000 times through the beta values in Elastic Net algorithm
            for iter in range(n_iters):
                𝛽x = coordinateDescent(x_train, y_train, 𝛽x, sum_sq, lamb, alpha)
            
            # Calulate MSE score for the model -- 
            # TODO: issue with MSE calualtion
            mse_score = score(x_test, y_test,  𝛽x)    
             
            # store the score with the tuning param combinations
            CV[i_fold, i_lambda, i_alpha] = mse_score


            # Store the coefficient vector
            𝛽[i_fold, i_lambda, i_alpha] = 𝛽x[:, 0]

        # Print out the mean CV MSE for lambda and alpha
        if verbose:
            print('lambda:{}; alpha:{}; CV MSE:{}'.format(lamb, alpha, np.mean(CV[:, i_lambda, i_alpha])))
                
          

### **Test bench for parameters**

In [ ]:
print(sum_sq)

In [ ]:
print(𝛽x)

In [ ]:
print(mse_score)

### **Retrain data with optimal lambda and alpha**

In [ ]:
#Retrain using all data with optimum lambda and alpha

#Calculate CV mean    
cv_mean = np.mean(CV, 0)

# find the best lambda and alpha  
best_λ_ind, best_alpha_index = np.where(cv_mean == np.amin(cv_mean))
best_λ = λ[best_λ_ind[0]]
best_alpha = α[best_alpha_index[0]]


# standardize features of x and center responses 
mean_x, std_x = np.mean(x, 0), np.std(x, 0)
x = standardize(x, mean_x, std_x)
y = centerResponses(y, np.mean(y))[:, None]
                                    

# Compute the sum of squares for each feature on the entire dataset
sum_sq = np.sum(x ** 2, 0)
                                    
# initialize 𝛽x coefficients
𝛽x = np.random.uniform(low = -1, high = 1, size = (X2, 1))

# Run coordent decent algorithm for best MSE 
for iter in range(n_iters):
    𝛽x = coordinateDescent(x, y, 𝛽x, sum_sq, best_λ, best_alpha)
    # print('Beta values updated test:',𝛽x) 

### **Test lambda and alpha**

- Best lambda shouild be 1.0
- Best alpha should be 1

In [ ]:
print('Best lambda:=', best_λ)

In [ ]:
print('Best alpha:=', best_alpha)

### **Output for Deliverable 1**

In [ ]:
# For each alpha, the coefficient values are plotted over the five folds as a function of lambda.
sns.set_theme()
sns.set_style("darkgrid", {"grid.color": ".5", "grid.linestyle": ":" })
𝛽μ  = np.mean(𝛽,0)
ŷ = df.columns
count = 0 # set itterator
t = TicToc()  # measure time of convergance
# create instance of class
for i_alpha, alpha in enumerate(α):
    count += 1 
    end_time = t.toc()
    plt.figure()
    plt.figure(figsize=(16, 10), dpi=70)
    print('Tuning parameter converged at = #{c} λ {} at alpha{α}\n'.format(np.log10(λ), c=count,  α=alpha)) 
    for i_beta in range(𝛽μ.shape[1]):
        plt.plot( np.log10(λ), 𝛽μ[:, i_alpha, i_beta], label = ŷ[i_beta])
    plt.legend(bbox_to_anchor = (1.05, 1), loc = 'upper right', title = 'Features')
    plt.xlabel('λ Tuning Params')
    plt.ylabel('Coefficient Values')
    plt.title('Alpha Value: {}'.format(alpha))
    plt.show()

### **Output for Deliverable 2**

In [ ]:
# Observe the CV MSE over values of lambda and alpha
plt.figure()
plt.figure(figsize=(16, 10), dpi=70)
for i_alpha, alpha in enumerate(α):
    std_error = np.std(CV[..., i_alpha], 0) / np.sqrt(k)
    plt.errorbar( np.log10(λ), np.mean(CV[..., i_alpha], 0), yerr = std_error,xuplims=True,label = str(alpha))
    plt.xlabel('Log base lambda')
    plt.ylabel('Cross Validation MSE')
    plt.legend(title = 'α')
plt.show()

### **Output for Deliverable 3**

In [ ]:
# lambda and alpha with lowest cv mse
print('Best lambda: {}; Best alpha: {}'.format(best_λ, best_alpha))

### **Output for Deliverable 4.1**

In [ ]:
# Lasso Implentation
# Plot the coefficient vectors for optimal lambda given alpha = 0 
# MSE cross valiadtion for alpha = 0
alpha0 = np.mean(CV[..., 0], 0)

# Get the index of lambda with lowest CV MSE
index = np.argmin(alpha0)
λx = λ[index]

# CV 5 K-fold mean vector coefficient for lambda and alpha
𝛽μ = np.mean(𝛽[:, index, 0, :], 0)

# Calulate and plot optimal lambda and alpha values
plt.figure()
plt.figure(figsize=(16, 10), dpi=70)
plt.scatter(𝛽x, 𝛽μ)
plt.plot(np.arange(-300, 475), np.arange(-300, 475), '--', color = 'g')
plt.xlabel('Elastic Net (lambda = {}, alpha = {})'.format(best_λ, best_alpha))
plt.ylabel('|L2|:= (lambda = {})'.format(λx))
plt.show()

####### Test #######
# print(alpha0)
# print(λ_i)
# print(𝛽μ )

### **Output for Deliverable 4.2**

In [ ]:
# Ridge Implentation
# Plot the coefficient vectors for optimal lambda given alpha = 0

# get cv mse given alpha = 1
alpha1 = np.mean(CV[..., -1], 0)

# Get the index of lambda with lowest CV MSE
λ_i = np.argmin(alpha1)
λx  = λ[λ_i]

# CV 5 K-fold mean vector coefficient for lambda and alpha
𝛽μ = np.mean(𝛽[:, λ_i, -1, :], 0)

# Calulate and plot optimal lambda and alpha values
plt.figure()
plt.figure(figsize=(16, 10), dpi=70)
plt.scatter(𝛽x, 𝛽μ)
plt.plot(np.arange(-300, 475), np.arange(-300, 475), '--', color = 'b')
plt.xlabel('Elastic Net (lambda = {}, alpha = {})'.format(best_λ, best_alpha))
plt.ylabel('L2 (lambda = {})'.format(λx))
plt.show()

####### Test #######
# print(alpha1)
# print(λ_i)
# print(𝛽μ )

# **Assignment 2 with ML libaries**
<hr>

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

from sklearn.exceptions import ConvergenceWarning
#from sklearn.utils._testing import ignore_warnings
import warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning) # To filter out the Convergence warning
warnings.filterwarnings('ignore', category=UserWarning)


from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV

# Scaling LibrariesL
from sklearn.preprocessing import StandardScaler

# Import packages for Measuring Model Perormance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer


from itertools import product


In [ ]:
# Target:
y= df['Balance'].to_numpy()

# Convert the Pandas dataframe to numpy ndarray for computational improvement
X = df.iloc[:,:-1]
X = X.to_numpy()



In [ ]:
# Define my tuning parameter values 𝜆:

learning_rates_λ = [1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]
# Define my L1 Ratio:
l1_ratio = [0, 1/5, 2/5, 3/5, 4/5, 1]

# Define our tunning rates
tuning_params = list(product(l1_ratio, learning_rates_λ))
tuning_params[:2]

In [ ]:
# Create Standarizing ObjectPackages:
standardization = StandardScaler()

# Strandardize 
n_observations = len(df)
variables = df.columns


# Standardize the Predictors (X)
Xst = standardization.fit_transform(X)

# Add a constanct to the predictor matrix
#Xst = np.column_stack((np.ones(n_observations),Xst))


# Save the original M and Std of the original data. Used for unstandardize
original_means = standardization.mean_

# we chanced standardization.std_ to standardization.var_**.5
originanal_stds = standardization.var_**.5




print("observations :", n_observations)
print("variables :", variables[:2])
print('original_means :', original_means)
print('originanal_stds :', originanal_stds)

In [ ]:
# Center y not using a library:
y_Mean = y.mean(axis = 0) # Original y mean

y_Centered =  y-y_Mean

print('Original y: ',y[:3])
print("mean of y :", y_Mean, "Std of y :", y.std(axis = 0))

print('Centered y: ',y_Centered[:3])
print("mean of y centered :", y_Centered.mean(axis = 0), "Std of y centered :", y_Centered.std(axis = 0))

print(y_Centered.shape)

In [ ]:
#let's first split it into train and test part
X_train, X_out_sample, y_train, y_out_sample = train_test_split(Xst, y_Centered, test_size=0.30, random_state=101) # Training and testing split

# Print Data size
print ("Train dataset sample size: {}".format(len(X_train)))
print ("Test dataset sample size: {}".format(len(X_out_sample)))

In [ ]:
from sklearn.linear_model import ElasticNet

L𝛽_per_λ=[] # set empty list

# Evaluate tuning parameters with Elastic Net penalty
for tuning_param in tuning_params:
        Library_ElasticNet=ElasticNet(alpha=tuning_param[1] , l1_ratio= tuning_param[0], max_iter=5000, tol=0.01)
        Library_ElasticNet.fit(X_train, y_train)
        c = np.array(Library_ElasticNet.coef_)
        c = np.append(tuning_param[1],c)
        c = np.append(tuning_param[0],c)
        L𝛽_per_λ.append(c)


In [ ]:
TunnedL𝛽_df=pd.DataFrame(L𝛽_per_λ)
TunnedL𝛽_df.columns=['Alpha', 'Lamba','Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education', 'Gender', 'Student', 'Married']
TunnedL𝛽_df.head()

## **Deliverable 6.1**  <a class="anchor" id="Deliverable_6.1"></a>
<h>

> Illustrate the effect of the tuning parameter on the inferred elastic net regression coefficients by generating six plots (one for each 𝛼 value) of nine lines (one for each of the 𝑝=9 features), with the 𝑦-axis as 𝛽̂
𝑗, 𝑗=1,2,…,9, and the 𝑥-axis the corresponding log-scaled tuning parameter value log10(𝜆) that generated the particular 𝛽̂
𝑗.

In [ ]:
plt.figure(figsize=(30,30))

plt.subplot(4, 2, 1)
plt.plot(TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(0)].iloc[:,1:2],TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(0)].iloc[:,2:])

plt.title('Effect of tunning on Coefficients alpha = 0')
plt.ylabel('Standardize Coefficients')

plt.xscale('log')
plt.legend(loc='best')
plt.legend(TunnedL𝛽_df.columns[2:])


plt.subplot(4, 2, 2)
plt.plot(TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(0.2)].iloc[:,1:2],TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(0.2)].iloc[:,2:])

plt.title('Effect of tunning on Coefficients alpha = 0.2')

plt.xscale('log')
plt.legend(loc='best')
plt.legend(TunnedL𝛽_df.columns[2:])


plt.subplot(4, 2, 3)

plt.plot(TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(0.4)].iloc[:,1:2],TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(0.4)].iloc[:,2:])

plt.title('Effect of tunning on Coefficients alpha = 0.4')
plt.ylabel('Standardize Coefficients')

plt.xscale('log')
plt.legend(loc='best')
plt.legend(TunnedL𝛽_df.columns[2:])


plt.subplot(4, 2, 4)

plt.plot(TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(0.6)].iloc[:,1:2],TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(0.6)].iloc[:,2:])

plt.title('Effect of tunning on Coefficients alpha = 0.6')

plt.xscale('log')
plt.legend(loc='best')
plt.legend(TunnedL𝛽_df.columns[2:])


plt.subplot(4, 2, 5)

plt.plot(TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(0.8)].iloc[:,1:2],TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(0.8)].iloc[:,2:])

plt.title('Effect of tunning on Coefficients alpha = 0.8')
plt.xlabel('Learning Rates λ')
plt.ylabel('Standardize Coefficients')

plt.xscale('log')
plt.legend(loc='best')
plt.legend(TunnedL𝛽_df.columns[2:])



plt.subplot(4, 2, 6)

plt.plot(TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(1)].iloc[:,1:2],TunnedL𝛽_df[TunnedL𝛽_df.Alpha.eq(1)].iloc[:,2:])

plt.title('Effect of tunning on Coefficients alpha = 1')
plt.xlabel('Learning Rates λ')

plt.xscale('log')
plt.legend(loc='best')
plt.legend(TunnedL𝛽_df.columns[2:])


# **Deliverable 6.2**  <a class="anchor" id="Deliverable_6.2"></a>
Illustrate the effect of the tuning parameters on the cross validation error by generating a plot of six lines (one for each l1_ratio value) with the y-axis as 
CV(5) error, and the x-axis the corresponding log-scaled tuning parameter value log10(λ) that generated the particular CV(5) error

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

#Define the model
Library_ElasticNet = ElasticNet()

# Create the Kfold:
cv_iterator = KFold(n_splits = 5, shuffle=True, random_state=101)
cv_score = cross_val_score(Library_ElasticNet, Xst, y_Centered, cv=cv_iterator, scoring='neg_mean_squared_error', n_jobs=1)
#print (cv_score)
#print ('Cv score: mean %0.3f std %0.3f' % (np.mean(np.abs(cv_score)), np.std(cv_score))) 

# define grid
Parm_grid = dict()
Parm_grid['alpha'] = learning_rates_λ
Parm_grid['l1_ratio'] = l1_ratio

In [ ]:
# Lets define search
GsearchCV = GridSearchCV(estimator = Library_ElasticNet, param_grid = Parm_grid, scoring = 'neg_mean_absolute_error', n_jobs=1, refit=True, cv=cv_iterator)
GsearchCV.fit(Xst, y_Centered)



In [ ]:
GCV_df = pd.concat([pd.DataFrame(GsearchCV.cv_results_["params"]),pd.DataFrame(GsearchCV.cv_results_["mean_test_score"], columns=["mean_test_score"])],axis=1)
#GCV_df.index=GCV_df['alpha']

GCV_df[:3]

In [ ]:
plt.figure(figsize=(25/2.54,15/2.54))

plt.plot( GCV_df[GCV_df.l1_ratio.eq(0)]["alpha"] , np.absolute( GCV_df[GCV_df.l1_ratio.eq(0)]["mean_test_score"] ), label='Avg_MSE_T alpha = 0')

plt.plot( GCV_df[GCV_df.l1_ratio.eq(0.2)]["alpha"] , np.absolute( GCV_df[GCV_df.l1_ratio.eq(0.2)]["mean_test_score"]), label='Avg_MSE_T alpha = 0.2')

plt.plot( GCV_df[GCV_df.l1_ratio.eq(0.4)]["alpha"] , np.absolute( GCV_df[GCV_df.l1_ratio.eq(0.4)]["mean_test_score"]), label='Avg_MSE_T alpha = 0.4')
plt.plot( GCV_df[GCV_df.l1_ratio.eq(0.6)]["alpha"] , np.absolute( GCV_df[GCV_df.l1_ratio.eq(0.6)]["mean_test_score"]), label='Avg_MSE_T alpha = 0.6')
plt.plot( GCV_df[GCV_df.l1_ratio.eq(0.8)]["alpha"] , np.absolute( GCV_df[GCV_df.l1_ratio.eq(0.8)]["mean_test_score"]), label='Avg_MSE_T alpha = 0.8')
plt.plot( GCV_df[GCV_df.l1_ratio.eq(1)]["alpha"] , np.absolute( GCV_df[GCV_df.l1_ratio.eq(1)]["mean_test_score"]), label='Avg_MSE_T alpha = 1')


plt.title('Effect of tunning on Coefficients')
plt.xlabel('Learning Rates λ')
plt.ylabel('Cross Validation MSE')

plt.xscale('log')
plt.legend(loc=0)

plt.show()

# **Deliverable 6.3**  <a class="anchor" id="Deliverable_6.3"></a>
Indicate the value of 𝜆 that generated the smallest CV(5) error

**Smallest CV with Library**

In [ ]:
print ('Best: ',GsearchCV.best_params_)
print ('Best CV mean squared error: %0.3f' % np.abs(GsearchCV.best_score_))

In [ ]:
GCV_df.sort_values(by=['mean_test_score'], ascending=False)[:1]


In [ ]:
# Alternative: sklearn.linear_model.ElasticNetCV
from sklearn.linear_model import ElasticNetCV


auto_EN = ElasticNetCV(alphas=learning_rates_λ, l1_ratio = l1_ratio, normalize=False,  n_jobs=1,  cv=cv_iterator)
auto_EN.fit(Xst, y_Centered)
print ('Best alpha: %0.5f' % auto_EN.alpha_)
print ('Best L1 ratio: %0.5f' % auto_EN.l1_ratio_)

# **Deliverable 6.4**  <a class="anchor" id="Deliverable_6.4"></a>
Given the optimal 𝜆, retrain your model on the entire dataset of 𝑁=400 observations and provide the estimates of the 𝑝=9 best-fit model parameters.


**Tunned with best alpha with Library**

In [ ]:
Library_ElasticNet_best=ElasticNet(alpha=GsearchCV.best_params_['alpha'] , l1_ratio= GsearchCV.best_params_['l1_ratio'], max_iter=1000, tol=0.1)
Library_ElasticNet_best.fit( Xst, y_Centered )

y_predM_best = Library_ElasticNet_best.predict(X_out_sample)
print ("Betas= ", Library_ElasticNet_best.coef_)

print("MSE = ",mean_squared_error(y_out_sample, y_predM_best))
print('R^2 Test', r2_score(y_out_sample, y_predM_best))

**lasso (𝛼=0 under optimal 𝜆 for 𝛼 =0)**

In [ ]:
Library_ElasticNet_best=ElasticNet(alpha=GsearchCV.best_params_['alpha'] , l1_ratio= 0, max_iter=1000, tol=0.1)
Library_ElasticNet_best.fit( Xst, y_Centered )

y_predM_best = Library_ElasticNet_best.predict(X_out_sample)
print ("Betas= ", Library_ElasticNet_best.coef_)

print("MSE = ",mean_squared_error(y_out_sample, y_predM_best))
print('R^2 Test', r2_score(y_out_sample, y_predM_best))

**ridge regression (𝛼=1 under optimal 𝜆 for 𝛼=1)**

In [ ]:
Library_ElasticNet_best=ElasticNet(alpha=GsearchCV.best_params_['alpha'] , l1_ratio= 1, max_iter=1000, tol=0.1)
Library_ElasticNet_best.fit( Xst, y_Centered )

y_predM_best = Library_ElasticNet_best.predict(X_out_sample)
print ("Betas= ", Library_ElasticNet_best.coef_)

print("MSE = ",mean_squared_error(y_out_sample, y_predM_best))
print('R^2 Test', r2_score(y_out_sample, y_predM_best))

Comapring the modeles with the optimal alpha (λ) and replacing the l1ratio (α) between 0 (lasso) and 1 (ridge) we are able to see that our model is more of a ridge model. Which is validated by the best_score_ (l1 ratio) from our CV Gridserach. The coefficient that is highly optimize is the Rating feature. Which is telling as balance, risk, and credit is highly correlated to this feature.